# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv') 

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
CustomerID     68584 non-null int64
FirstName      68584 non-null object
LastName       68584 non-null object
SalesID        68584 non-null int64
ProductID      68584 non-null int64
ProductName    68584 non-null object
Quantity       68584 non-null int64
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
grouped = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": "sum"})
grouped.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [31]:
pivot = pd.pivot_table(grouped, values="Quantity", index="ProductName", columns="CustomerID", fill_value=0, dropna=False)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [244]:
dist = pdist(pivot.T, metric="cosine")
dist

array([0.88678126, 0.77464789, 0.83977196, ..., 0.84074449, 0.88165473,
       0.83037577])

In [245]:
sqForm = squareform(dist)
sqForm

array([[0.        , 0.88678126, 0.77464789, ..., 0.9223069 , 0.88864251,
        0.89702036],
       [0.88678126, 0.        , 0.86162154, ..., 0.8612137 , 0.90053882,
        0.89652438],
       [0.77464789, 0.86162154, 0.        , ..., 0.85756266, 0.80203113,
        0.89702036],
       ...,
       [0.9223069 , 0.8612137 , 0.85756266, ..., 0.        , 0.84074449,
        0.88165473],
       [0.88864251, 0.90053882, 0.80203113, ..., 0.84074449, 0.        ,
        0.83037577],
       [0.89702036, 0.89652438, 0.89702036, ..., 0.88165473, 0.83037577,
        0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [246]:
newDf = pd.DataFrame(1/(1 + sqForm), index=pivot.columns, columns=pivot.columns)
topCust = newDf[33].sort_values(ascending=False)[1:].nlargest(5)
topCustList = topCust.index.tolist()
print(topCustList)

[27672, 45549, 3531, 76532, 60862]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [247]:
top5Cust = grouped.loc[topCustList]
top5Cust

Quantity
CustomerID ProductName                            
3531       Baking Powder                         1
           Bar Mix - Pina Colada, 355 Ml         1
           Bay Leaf                              1
           Beans - Kidney White                  1
           Beef - Chuck, Boneless                1
...                                            ...
76532      Tia Maria                            20
           Tomato - Tricolor Cherry             20
           Veal - Osso Bucco                    20
           Veal - Sweetbread                    20
           Wine - Gato Negro Cabernet           20

[354 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [248]:
top5CustAgg = top5Cust.groupby("ProductName").agg({"Quantity": "sum"})
top5CustAgg.sort_values("Quantity", ascending=False)

,Quantity
ProductName,
Knife Plastic - White,64
Phyllo Dough,64
Cake - Box Window 10x10x2.5,52
"Soup - Campbells, Beef Barley",52
Onions - Cippolini,48
...,...
Chef Hat 20cm,1
"Turnip - White, Organic",1
Veal - Eye Of Round,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [249]:
customer33 = pivot[[33]]
customer33.head()

CustomerID,33
ProductName,
Anchovy Paste - 56 G Tube,0
"Appetizer - Mini Egg Roll, Shrimp",0
Appetizer - Mushroom Tart,0
Appetizer - Sausage Rolls,0
Apricots - Dried,1


In [250]:
customer33Sales = customer33.merge(top5CustAgg, left_on="ProductName", right_on="ProductName")
customer33Sales = customer33Sales[customer33Sales[33] == 0].sort_values(["Quantity"], ascending=False)

In [251]:
customer33Sales.nlargest(5, "Quantity")

,33,Quantity
ProductName,,
Knife Plastic - White,0,64
"Soup - Campbells, Beef Barley",0,52
Onions - Cippolini,0,48
Tea - Herbal Sweet Dreams,0,45
Banana Turning,0,40


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [252]:
recomendations = {}
for customer in pivot.columns:
    newDf = pd.DataFrame(1/(1 + sqForm), index=pivot.columns, columns=pivot.columns)
    topCust = newDf[customer].sort_values(ascending=False)[1:].nlargest(5)
    topCustList = topCust.index.tolist()
    top5Cust = grouped.loc[topCustList]
    top5CustAgg = top5Cust.groupby("ProductName").agg({"Quantity": "sum"})
    top5CustAgg.sort_values("Quantity", ascending=False)
    customerNum = pivot[[customer]]
    customerNumSales = customerNum.merge(top5CustAgg, left_on="ProductName", right_on="ProductName")
    customerNumSales = customerNumSales[customerNumSales[customer] == 0].sort_values(["Quantity"], ascending=False)
    recomendations[f"{customer}"] = customerNumSales.nlargest(5, "Quantity")[customer].index.tolist()

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [229]:
recomenDf = pd.DataFrame.from_dict(recomendations)
recomenDfT = recomenDf.T
recomenDfT.index.names = ["CustomerID"]
recomenDfT.columns=["Item 1", "Item 2", "Item 3", "Item 4", "Item 5"]
recomenDfT.head()

,Item 1,Item 2,Item 3,Item 4,Item 5
CustomerID,,,,,
33,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Wine - Ej Gallo Sierra Valley,Butter - Unsalted,Yoghurt Tubes
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Wanton Wrap,Bay Leaf,Pork - Kidney
264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,"Veal - Inside, Choice",Bread - Italian Roll With Herbs,Fondant - Icing
356,"Veal - Inside, Choice",Lamb - Ground,Butter - Unsalted,Sardines,Beef - Inside Round
412,"Sauce - Gravy, Au Jus, Mix",Sprouts - Baby Pea Tendrils,Olive - Spread Tapenade,Wine - Blue Nun Qualitatswein,Cheese - Cambozola


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [242]:
recomenDf = pd.DataFrame.from_dict(recomendations)
recomenDfT = recomenDf.T
recomenDfT.index.names = ["CustomerID"]
recomenDfT.columns=["Item 1", "Item 2", "Item 3", "Item 4", "Item 5"]
recomenDfT.head()

,Item 1,Item 2,Item 3,Item 4,Item 5
CustomerID,,,,,
33,"Oranges - Navel, 72",Chocolate - Dark,Pecan Raisin - Tarts,Bay Leaf,Wine - Charddonnay Errazuriz
200,Soup - Campbells Bean Medley,Bay Leaf,Pork - Kidney,Scampi Tail,Muffin - Carrot Individual Wrap
264,"Veal - Inside, Choice",Bread - Italian Roll With Herbs,Muffin - Carrot Individual Wrap,Wine - Two Oceans Cabernet,Black Currants
356,Butter - Unsalted,"Veal - Inside, Choice","Nut - Chestnuts, Whole","Beets - Candy Cane, Organic",Raspberries - Fresh
412,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Table Cloth 81x81 White,"Pepper - Black, Whole",Butter - Unsalted


In [253]:
recomenDf = pd.DataFrame.from_dict(recomendations)
recomenDfT = recomenDf.T
recomenDfT.index.names = ["CustomerID"]
recomenDfT.columns=["Item 1", "Item 2", "Item 3", "Item 4", "Item 5"]
recomenDfT.head()

,Item 1,Item 2,Item 3,Item 4,Item 5
CustomerID,,,,,
33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,Banana Turning
200,Longos - Grilled Salmon With Bbq,Snapple Lemon Tea,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop
264,Pickerel - Fillets,"Water - Mineral, Natural",Snapple - Iced Tea Peach,Wine - Ej Gallo Sierra Valley,French Pastry - Mini Chocolate
356,Bread - English Muffin,Olive - Spread Tapenade,Bagel - Plain,Pork - Hock And Feet Attached,Oil - Shortening - All - Purpose
412,"Salsify, Organic",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Gatorade - Xfactor Berry
